In [48]:
import pandas as pd
"""!pip install bs4 
!pip install sklearn
!pip install nltk
!pip install gensim
!pip install lxml"""
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import lxml
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm
from gensim.models import doc2vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

In [49]:
df = pd.read_csv("/content/mbti_1.csv")
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [50]:
from nltk.corpus import stopwords
nltk.download('stopwords')


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    
    text = text.lower() # lowercase text
    
    
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:


"""for k in range(0,len(df.index)):
  print(k)
  print(df["posts"][k])
  df["posts"][k] = clean_text(df["posts"][k])"""

df["posts"] = df["posts"].apply(clean_text)
df

,type,posts
0,INFJ,http wwwyoutubecom watchvqscwe3krw http 41medi...
1,ENTP,finding lack posts alarming ex boring position...
2,INTP,ood one _____ https wwwyoutubecom watchvfibolw...
3,INTJ,ear enjoyed conversation day soteric gabbing n...
4,ENTJ,oure fired hats another silly misconception ha...
...,...,...
8670,ISFP,https wwwyoutubecom watchvt8ed_h908 x always t...
8671,ENFP,oif thread already exists someplace else http ...
8672,INTP,many questions things would take purple pill i...
8673,INFP,conflicted right comes wanting children honest...


In [52]:
x= df["posts"]
y=df["type"]


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)


In [53]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [54]:
X_train = label_sentences(x_train, 'Train')
X_test = label_sentences(x_test, 'Test')
all_data = X_train + X_test
#all_data[0:5]

In [55]:
model_dbow = doc2vec.Doc2Vec(dm=0, vector_size=100, negative=5, min_count=1, alpha=0.065, 
                     min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 8675/8675 [00:00<00:00, 1499076.60it/s]


In [56]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), 
                     total_examples=len(all_data), 
                     epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 8675/8675 [00:00<00:00, 2640463.51it/s]


In [57]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [58]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 100, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 100, 'Test')

In [59]:
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

def nb_model(x_train,x_test,y_train,y_test):
  

  classifier = GaussianNB()  
  classifier.fit(x_train, y_train) 
  y_pred = classifier.predict(x_test)

 
  return 'accuracy %s' % accuracy_score(y_pred, y_test)

In [60]:
print(nb_model(train_vectors_dbow,test_vectors_dbow,y_train,y_test))


accuracy 0.35266999615827893
